# Introduction

- Previous: [02-do-review-code.ipynb](./02-do-review-code.ipynb)
- First: [00-do-setup-domain.ipynb](./00-do-setup-domain.ipynb)

# Prerequisites
You have ran the [02-do-review-code.ipynb](./02-do-review-code.ipynb) and the submitted code is approved by both `canada-domain` and `italy-domain`.

In [ ]:
# syft absolute
import syft as sy
from syft.abstract_server import ServerType

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082
CANADA_ENCLAVE_PORT = 9083

In [ ]:
# Launch the domain servers we setup in the previous notebook
canada_server = sy.orchestra.launch(
    name="canada-domain", port=CANADA_DOMAIN_PORT, dev_mode=True
)
italy_server = sy.orchestra.launch(
    name="italy-domain", port=ITALY_DOMAIN_PORT, dev_mode=True
)
canada_enclave = sy.orchestra.launch(
    name="canada-enclave",
    server_type=ServerType.ENCLAVE,
    port=CANADA_ENCLAVE_PORT,
    dev_mode=True,
    create_producer=True,
    n_consumers=3,
    reset=True,  # * Reset the enclave each time for ease in development
)

In [ ]:
ds_canada_client = canada_server.login(
    email="sheldon@caltech.edu", password="changethis"
)
ds_italy_client = italy_server.login(email="sheldon@caltech.edu", password="changethis")

In [ ]:
project = ds_canada_client.api.services.project.get_by_name("Census Matching")
project

In [ ]:
assert project.pending_requests == 0

In [ ]:
code = project.code[0]

In [ ]:
code.setup_enclave()

In [ ]:
code.request_asset_transfer()

In [ ]:
code.request_execution()

In [ ]:
result = code.get_result()
print(result)

In [ ]:
# Or you can call all of the above in one line using the following
result = code.orchestrate_enclave_execution()
print(result)

# Cleanup local domain servers

In [ ]:
if canada_server.deployment_type.value == "python":
    canada_server.land()

if italy_server.deployment_type.value == "python":
    italy_server.land()

if canada_enclave.deployment_type.value == "python":
    canada_enclave.land()